# Installing Libraries (Python version >= 3.8)

In [ ]:
import sys
version = sys.version_info
print(version)
assert version.major == 3 and version.minor >= 8

In [ ]:
!python -m pip install -U numpy==1.23.5 pandas==1.5.3 scikit-learn==1.2.2 matplotlib==3.7.4 seaborn==0.13.2

# Downloading/Visualizing Dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets, model_selection

dataset = datasets.fetch_california_housing()
X = pd.DataFrame(data=dataset.data, columns=dataset.feature_names)
y = pd.Series(data=dataset.target, name="target")

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=0)

display(X.head())
print("samples: {}; features: {}".format(*X.shape))

In [ ]:
display(y.head())
print("samples: {}".format(*y.shape))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

corr_matrix = pd.concat([X, y], axis=1).corr()

plt.figure(figsize=(9, 6))
plt.title("Correlation Matrix")
sns.heatmap(corr_matrix, annot=True, square=True, cmap="Blues", fmt=".2f", linewidths=.5)
# plt.savefig("california_housing_corr_matrix.png")

# Training Model

In [ ]:
class LinearRegression:

    def __init__(self, alpha: float = 1e-7, eps: float = 1e-4) -> None:
        self.alpha = alpha  # Learning rate for gradient descent
        self.eps = eps  # Threshold of convergence

    def fit(self, X: pd.DataFrame, y: pd.Series) -> "LinearRegression":
        """Train the model. Optimization method is gradient descent.

        :param X: The feature values of the training data.
        :param y: The target values of the training data.
        :return: The trained model.
        """
        self._m = X.shape[0]  # The number of samples
        num_features = X.shape[1]  # The number of features

        self._theta = np.zeros(num_features)  # Parameters (weight) of the model (without bias)
        self._theta0 = np.zeros(1)  # Bias of the model

        self._error_values = []  # The output values of the cost function in each iteration
        self._grad_values = []  # Gradient values in each iteration
        self._iter_counter = 0  # The counter of iterations
        
        error = self.J(X, y)  # The initial output value of the cost function with random parameters
        diff = 1.0  # The difference between the previous and the current output values of the cost function

        # Repeat until convergence
        while diff > self.eps:
            # Update the parameters by gradient descent
            y_pred = self.predict(X)  # Predict the target values with the current parameters
            grad = (1 / self._m) * np.dot(y_pred - y, X)  # Calculate the gradient using the formula
            self._theta -= self.alpha * grad  # Update the parameters
            self._theta0 -= (1 / self._m) * np.sum(y_pred - y)  # Update the bias

            # Print the current status
            _error = self.J(X, y)  # Compute the error with the updated parameters
            diff = abs(error - _error)  # Compute the difference between the previous and the current error
            error = _error  # Update the error
            self._error_values.append(error)
            self._grad_values.append(grad.sum())
            self._iter_counter += 1
            print(f"[{self._iter_counter}] error: {error}, diff: {diff}, grad: {grad.sum()}")
        print(f"Convergence in {self._iter_counter} iterations.")
        return self

    def predict(self, X: pd.DataFrame) -> np.ndarray:
        """Predict the target values using the hypothesis function.

        :param X: The feature values of the data.
        :return: The predicted target values.
        """
        # Pass the bias and the parameters to the hypothesis function
        theta = np.concatenate([self._theta0, self._theta])
        return self.h(X, theta)

    def h(self, X: pd.DataFrame, theta: np.ndarray) -> np.ndarray:
        """Hypothesis function.

        :param X: The feature values of the data.
        :param theta: The parameters (weight) of the model.
        :return: The predicted target values.
        """
        # theta[0] is bias and theta[1:] is parameters
        return np.dot(X, theta[1:].T) + theta[0]

    def J(self, X: pd.DataFrame, y: pd.Series) -> float:
        """Cost function. Mean squared error (MSE).

        :param X: The feature values of the data.
        :param y: The target values of the data.
        :return: The error value.
        """
        y_pred = self.predict(X)  # Predict the target values with the current parameters
        return (1 / (2 * self._m)) * np.sum((y_pred - y) ** 2)  # Compute the error using the formula

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
fig = plt.figure(figsize=(15, 5))

ax = fig.add_subplot(1, 2, 1)
ax.set_title("MSE")
ax.set_ylabel("Error")
ax.set_xlabel("Iteration")
ax.plot(np.arange(model._iter_counter), model._error_values, color="b")

ax = fig.add_subplot(1, 2, 2)
ax.set_title("Gradient")
ax.set_ylabel("Gradient")
ax.set_xlabel("Iteration")
ax.plot(np.arange(model._iter_counter), model._grad_values, color="r")

plt.show()

# Evaluating Model

In [ ]:
from sklearn.metrics import mean_squared_error

y_train_pred = model.predict(X_train)
print(f"MSE for train data: {mean_squared_error(y_train, y_train_pred)}")
y_test_pred = model.predict(X_test)
print(f"MSE for test data: {mean_squared_error(y_test, y_test_pred)}")

# Regularization

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(alpha=1.0)  # L2 norm
ridge.fit(X_train, y_train)

print(f"MSE for train data: {mean_squared_error(y_train, ridge.predict(X_train))}")
print(f"MSE for test data: {mean_squared_error(y_test, ridge.predict(X_test))}")

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=1.0)  # L1 norm
lasso.fit(X_train, y_train)

print(f"MSE for train data: {mean_squared_error(y_train, lasso.predict(X_train))}")
print(f"MSE for test data: {mean_squared_error(y_test, lasso.predict(X_test))}")